## Import Library

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, Dropout, Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import glob
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
working_dir = "/Users/pushpendrammishra/Desktop/Denoising"
noisy_train_path = working_dir + "/Train/low"
clean_train_path = working_dir + "/Train/high"
noisy_test_path = working_dir + "/test/low"
predicted_dir = working_dir + '/test/predicted/'


## Get a sorted list of images from the subdirectory specified

In [ ]:
# Ensure predicted directory exists
os.makedirs(predicted_dir, exist_ok=True)

noisy_train_image = sorted(glob.glob(os.path.join(noisy_train_path, '*')))
clean_train_image = sorted(glob.glob(os.path.join(clean_train_path, '*')))


In [ ]:
noisy_test_image = sorted(glob.glob(os.path.join(noisy_test_path, '*')))

In [ ]:
noisy_train_image[:7]

In [ ]:
len(noisy_train_image)

In [ ]:
clean_train_image[:7]

In [ ]:
len(clean_train_image)

## Preprocessing

In [ ]:
img=Image.open(os.path.join(noisy_train_path+'/100.png'))
img.size

In [ ]:
def image_preprocessing(path):
    img = Image.open(path)
    img = img.resize((600, 400))
    img = img.convert("L")
    img = np.asarray(img, dtype="float32") / 255.0
    img = np.reshape(img, (400, 600, 1))
    return img

In [ ]:
noised_train = [image_preprocessing(f) for f in noisy_train_image]
cleaned_train = [image_preprocessing(f) for f in clean_train_image]

 ## Noised train image Vs Denoised train image

In [ ]:
# Plot some training images
plt.figure(figsize=(10, 20))
for i in range(0, 4, 2):
    plt.subplot(4, 2, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(noised_train[i][:, :, 0], cmap='gray')
    plt.title('Image with Noise:')
    
    plt.subplot(4, 2, i + 2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(cleaned_train[i][:, :, 0], cmap='gray')
    plt.title('Denoised image:')

plt.show()

## Split Dataset

In [ ]:
x_train = np.asarray(noised_train)
y_train = np.asarray(cleaned_train)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20)

In [ ]:
x_train.shape

## Model building

In [ ]:
def autoencoder():
    # Input layer
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(400, 600, 1,))) 
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    return model

In [ ]:
model = autoencoder()
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=["accuracy"])
 
callback = EarlyStopping(monitor='val_loss', patience=3)
 
model.fit(x_train, y_train, epochs=300, batch_size=53, validation_data=(x_val, y_val),callbacks=[callback] )
 


In [ ]:
model.save(working_dir+'/model.h5')

## Prediction

In [ ]:
# Load and preprocess test images
noised_test = [image_preprocessing(f) for f in noisy_test_image]
x_test = np.asarray(noised_test)

In [ ]:
# Predict denoised images
pred = model.predict(x_test, batch_size=16)

In [ ]:
denoised_test_image=sorted(glob.glob(os.path.join(working_dir+'/test/Ground_truth', '*')))
denoised_test=[image_preprocessing(f) for f in denoised_test_image]
denoised_test=np.asarray(denoised_test)

In [ ]:
psnr_values = []
# Ensure predicted directory exists
os.makedirs(predicted_dir, exist_ok=True)
for i in range(len(x_test)):
    original = np.clip(denoised_test[i] * 255.0, 0, 255)  # Clip values to [0, 255]
    denoised = np.clip(pred[i] * 255.0, 0, 255)   # Clip values to [0, 255]
    
    psnr_value = psnr(original, denoised, data_range=255)  # Specify data_range
    psnr_values.append(psnr_value)
    
#     print(f'PSNR for image {noisy_test_image[i]}: {psnr_value:.2f} dB')
    
    denoised_img = Image.fromarray(denoised.squeeze().astype(np.uint8))
    denoised_img.save(os.path.join(predicted_dir, os.path.basename(noisy_test_image[i])))

In [ ]:
mean_psnr = np.mean(psnr_values)
print(f"Mean PSNR: {mean_psnr:.2f}")

In [ ]:
# Plot some examples with their PSNR values
plt.figure(figsize=(15, 25))
for i in range(0, 8, 2):
    if i >= len(x_test):
        break
    plt.subplot(4, 2, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[i][:, :, 0], cmap='gray')
    plt.title('Test Image with Noise')
    
    plt.subplot(4, 2, i + 2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(pred[i][:, :, 0], cmap='gray')
    plt.title(f'Denoised by Autoencoder - PSNR: {psnr_values[i]:.2f} dB')

plt.show()
